In [1]:
from __future__ import absolute_import
from matplotlib import pyplot as plt
from numpy.lib.function_base import _DIMENSION_NAME, select
from tensorflow.python.framework.tensor_conversion_registry import get
from tensorflow.python.ops.gen_math_ops import exp
from tensorflow.python.ops.gen_nn_ops import MaxPool
import os
import tensorflow as tf
import numpy as np
import random
import math
from preprocess import get_data
from model import Model
from one_hot import decode, encode
from extract import Extractor


In [2]:

def train(model, train_inputs, train_labels):
    '''
    Trains the model on all of the inputs and labels for one epoch. 
    
    :param model: the initialized model to use for the forward pass and backward pass
    :param train_inputs: train inputs (all inputs to use for training) 
    :param train_labels: train labels (all labels to use for training)
    :return: loss list and accuracy
    '''

    #loop through data in batches
    accuracy = 0
    j = 0
    for i in range(0, train_inputs.shape[0], model.batch_size):

        if(i + model.batch_size > train_inputs.shape[0]):
            break

        #maybe change the shape of the inputs depending on what they are
        batch_input = train_inputs[i : i + model.batch_size]
        batch_label = train_labels[i : i + model.batch_size]

        #calculated logit and loss
        with tf.GradientTape() as tape:
            logits = model.call(batch_input)
            loss = model.loss(logits, batch_label)
            
            accuracy += model.accuracy(logits, batch_label)
            model.loss_list.append(loss)

        #gets the gradient
        gradients = tape.gradient(loss, model.trainable_variables)

        #applies the optimizer
        model.optimization.apply_gradients(zip(gradients, model.trainable_variables))

        j+=1

    return model.loss_list, accuracy/j



def test_characters(model, test_inputs, test_labels):
    """
    Tests the model on the test inputs and labels.
    :param test_inputs: test data (all images to be tested), 
    :param test_labels: test labels (all corresponding labels),
    :return: test accuracy - testing accuracy
    """
    accuracy = 0
    num_batches = 0

    #loop through the data in batches
    for i in range(0, len(test_inputs), model.batch_size):

        if(i + model.batch_size > len(test_inputs)):
            break

        #get the batched inputs and labels
        batch_input = test_inputs[i : i + model.batch_size]
        batch_label = test_labels[i : i + model.batch_size]

        #get logits and calculated accuracy
        logits = model.call(batch_input)
        accuracy += model.accuracy(logits, batch_label)

        num_batches+=1

    return accuracy/num_batches


def test_expressions(model, test_inputs, test_labels, extractor1):
    """
    Tests the model on the test inputs and labels fo full mathematical 
    expressions.
    
    :param test_inputs: test data (all images to be tested), 
    :param test_labels: test labels (all corresponding labels),
    :return: test accuracy - testing accuracy
    """
    accuracy = 0

    #loop through the data in batches

    for i in range(len(test_inputs)):

        # each expression acts as a batch
        expression_input = test_inputs[i]
        expression_input = np.array(expression_input)
        #reshape each expression for testing
        expression_input = np.reshape(expression_input, (-1,32,32,1))
        expression_label = test_labels[i]

        #get logits and calculated accuracy
        logits = model.call(expression_input)

        #create a fucntion that measures accuracy for expressions
        print("Predicted label: ", decode_expression(logits, extractor1))
        print("True Label: ", decode_expression(expression_label, extractor1))
        accuracy += model.accuracy(logits, expression_label)

    return accuracy/len(test_inputs)

def visualize_loss(losses): 
    """
    Uses Matplotlib to visualize the losses of our model.
    :param losses: list of loss data stored from train. Can use the model's loss_list 
    field 
    :return: doesn't return anything, a plot should pop-up 
    """
    x = [i for i in range(len(losses))]
    plt.plot(x, losses)
    plt.title('Loss per batch')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.show()  


def decode_expression(probabilities, extractor1):
    """
    Decodes the expression from probabilities into actual math symbols
    param probabilities: the probabilities returned by the model
    param extractor: the extractor that was used to preprocess the data
    """
    predicted_labels = np.argmax(probabilities, axis=1)
    output_symbols = []
    for i in range(len(predicted_labels)):
        symbol = decode(predicted_labels[i], extractor1.classes)
        output_symbols += symbol
    return output_symbols


def main():

    #get and load data
    train_inputs, train_labels, test_inputs, test_labels, test_char_inputs, test_char_labels, extractor1 = get_data()
    print("Preprocessing Completed!")

    model = Model()

    #train model
    for i in range(10):
        loss_list, accuracy = train(model, train_inputs, train_labels)
        print("Epoch",i , " ", accuracy)
        print("Loss:", tf.reduce_mean(model.loss_list))
    #visualize_loss(loss_list)

    print("Accuracy for Training", accuracy)
    
    # test model on characters

    acc_1 = test_characters(model, test_char_inputs, test_char_labels)
     
    print("Accuracy for testing characters: ", acc_1)

    # test model from expression
    acc = test_expressions(model, test_inputs, test_labels, extractor1)

    print("Accuracy for testing expression", acc)
    pass




In [3]:
if __name__ == '__main__':
    main()

Preprocessing Completed!


C:\Users\Anubhab Bose\anaconda3\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0   tf.Tensor(0.19002989, shape=(), dtype=float32)
Loss: tf.Tensor(3.397055, shape=(), dtype=float32)
Epoch 1   tf.Tensor(0.66925365, shape=(), dtype=float32)
Loss: tf.Tensor(2.4091065, shape=(), dtype=float32)
Epoch 2   tf.Tensor(0.8169552, shape=(), dtype=float32)
Loss: tf.Tensor(1.8455932, shape=(), dtype=float32)
Epoch 3   tf.Tensor(0.893194, shape=(), dtype=float32)
Loss: tf.Tensor(1.4826012, shape=(), dtype=float32)
Epoch 4   tf.Tensor(0.92477626, shape=(), dtype=float32)
Loss: tf.Tensor(1.2398401, shape=(), dtype=float32)
Epoch 5   tf.Tensor(0.9404777, shape=(), dtype=float32)
Loss: tf.Tensor(1.0663874, shape=(), dtype=float32)
Epoch 6   tf.Tensor(0.95002997, shape=(), dtype=float32)
Loss: tf.Tensor(0.93752396, shape=(), dtype=float32)
Epoch 7   tf.Tensor(0.9527164, shape=(), dtype=float32)
Loss: tf.Tensor(0.8392764, shape=(), dtype=float32)
Epoch 8   tf.Tensor(0.9591643, shape=(), dtype=float32)
Loss: tf.Tensor(0.7605455, shape=(), dtype=float32)
Epoch 9   tf.Tensor(0.971

Predicted label:  ['y', 'a', '2', '+', 'b', '=', 'x']
True Label:  ['y', 'a', '2', '+', 'b', '=', 'x']
Predicted label:  ['a', '\\', 'i', 'n', 't', 'B', '1', '1', 'y', ')', '\\', 'a', 'l', 'p', 'h', 'a', 'y', '+', '3', 'B', '2', '(', 'y', '1', '\\', 'a', 'l', 'p', 'h', 'a', 'y', '=', '\\', 'a', 'l', 'p', 'h', 'a', 'x', '\\', 'a', 'l', 'p', 'h', 'a', 'x']
True Label:  ['a', '\\', 'i', 'n', 't', '\\', 'b', 'e', 't', 'a', '1', '(', 'y', ')', 'd', 'y', '+', '\\', 'i', 'n', 't', '\\', 'b', 'e', 't', 'a', '2', '(', 'y', ')', 'd', 'y', '=', '\\', 'a', 'l', 'p', 'h', 'a', 'x', '\\', 'a', 'l', 'p', 'h', 'a', 'x']
Predicted label:  ['n', 'x', '=', '\\', 'p', 'm', 'n']
True Label:  ['n', 'x', '=', '\\', 'p', 'm', '\\', 'p', 'i']
Predicted label:  ['-', 'n', 'm', '6', '=', '2']
True Label:  ['-', 'n', '\\', 'p', 'i', 't', '=', 'x']
Predicted label:  ['1', '3', '7', '+', '6', '2', '-', '4', '4', '\\', 'n', 'e', 'q']
True Label:  ['1', '9', '7', '+', '6', '2', '-', '7', '4', '\\', 'n', 'e', 'q']
Pre

Predicted label:  ['7', '(', 'x', ')', '1', '-', 'b', '-', 'a', '=']
True Label:  ['f', '(', 'x', ')', '1', '-', 'b', '-', 'a', '=']
Predicted label:  ['z', '3', '2', '\\', 's', 'q', 'r', 't', '2', 'n', '2', '-', '2', '[', '-', '\\', 's', 'q', 'r', 't', '2', '=', '=']
True Label:  ['z', '3', '2', '-', '2', '\\', 's', 'q', 'r', 't', '2', '-', '2', '1', '-', '\\', 's', 'q', 'r', 't', '2', '=', '=']
Predicted label:  ['a', '1', '\\', 's', 'q', 'r', 't', '1', '-', '(', '-', ')', '(', '1', '-', '\\', 's', 'q', 'r', 't', '3', ')', '=', '1', '1', '1', 'a', '1']
True Label:  ['a', '1', 'r', '1', '-', '(', '-', ')', '(', '1', '-', 'r', '3', ')', '=', '1', 'r', 'r', '2', 'r']
Predicted label:  ['4', 'a', '4', '5', '=']
True Label:  ['\\', 'p', 'h', 'i', 'a', '\\', 'p', 'h', 'i', 'b', '=']
Predicted label:  ['(', '(', '1', '4', 'k', '+', '1', '(', '9', ')', '-', '(', '7', '1', '6', '1', '8', '6', ')', ')', '+', '1', '6', '4', '7', '3', '2', '3', '\\', 'd', 'i', 'v']
True Label:  ['(', '(', '1', '

...

Predicted label:  ['a', '(', 'n', ')', '=', '\\', 's', 'u', 'm', 'n', 'k', '=', '1', '(', '-', '(', ')', 'n', '-', '\\', 'a', 'l', 'p', 'h', 'a', 'k', '!']
True Label:  ['a', '(', 'n', ')', '=', '\\', 's', 'u', 'm', 'n', 'k', '=', '1', '(', '-', '1', ')', 'n', '-', 'k', 'k', '!']
Predicted label:  ['\\', 'i', 'n', 't', '2', '1', '(', '-', 'x', '2', '-', '1', 'x', '2', ')', 'e', 'x', '+', '-', '1', 'x', 'd', 'x']
True Label:  ['\\', 'i', 'n', 't', '2', '1', '(', '-', 'x', '2', '-', '1', 'x', '2', ')', 'e', 'x', '+', '-', '1', 'x', 'd', 'x']
Predicted label:  ['\\', 's', 'i', 'n', '(', '\\', 'a', 'l', 'p', 'h', 'a', '+', '\\', 'b', 'e', 't', 'a', ')', '\\', 's', 'i', 'n', '(', '\\', 'a', 'l', 'p', 'h', 'a', '\\', 's', 'q', 'r', 't', '\\', 'b', 'e', 't', 'a', ')', '=', '\\', 's', 'i', 'n', '2', '\\', 'a', 'l', 'p', 'h', 'a', '-', '\\', 's', 'i', 'n', '2', '\\', 'b', 'e', 't', 'a']
True Label:  ['\\', 's', 'i', 'n', '(', '\\', 'a', 'l', 'p', 'h', 'a', '+', '\\', 'b', 'e', 't', 'a', ')', '\

Predicted label:  ['(', 'x', '3', '-', 'x', '2', '-', 'x', ')', '(', '2', 'x', '-', '7', ')']
True Label:  ['(', 'x', '3', '-', 'x', '2', '-', 'x', ')', '(', '2', 'x', '-', '7', ')']
Predicted label:  ['e', '2', '1', '+', 'e', '3', '2', '+', 'e', '4', '3', '+', 'e', '5', 'y']
True Label:  ['e', '2', '1', '+', 'e', '3', '2', '+', 'e', '4', '3', '+', 'e', '5', '4']
Predicted label:  ['\\', 's', 'i', 'n', 'x', '-', '\\', 's', 'i', 'n', 'y', '-', '\\', 's', 'i', 'n', '(', 'x', '-', 'y', ')']
True Label:  ['\\', 's', 'i', 'n', 'x', '-', '\\', 's', 'i', 'n', 'y', '-', '\\', 's', 'i', 'n', '(', 'x', '-', 'y', ')']
Predicted label:  ['\\', 's', 'q', 'r', 't', '2', '(', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '(', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', ')']
True Label:  ['\\', 's', 'q', 'r', 't', '2', '(', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x

Predicted label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '2', '-', '(', '1', '-', '\\', 'c', 'o', 's', 'x', ')', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')', 'x', '2', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')']
True Label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '(', '1', '-', '\\', 'c', 'o', 's', 'x', ')', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')', 'x', '2', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')']
Predicted label:  ['\\', 'i', 'n', 't', '(', '2', 'x', '-', '3', 'e', 'x', ')', 'd', 'x']
True Label:  ['\\', 'i', 'n', 't', '(', '2', 'x', '-', '3', 'e', 'x', ')', 'd', 'x']
Predicted label:  ['-', '1', 'a', 'F', '(', 'a', 'x', '+', 'b', ')', '+', 'C']
True Label:  ['-', '1', 'a', 'F', '(', 'a', 'x', '+', 'b', ')', '+', 'C']
Predicted label:  ['\\', 'i', 'n', 't', '-', '\\', 'p', 'i', '2', '0', '\\', '{', '(', '\\', 'c', 'o', 's', 'x', '+', 'e', 'x', ')', '-', '(', 'e', 'k', '-', '

Predicted label:  ['e', '2', '1', '+', 'e', '3', '2', '+', 'e', '4', '3', '+', 'e', '5', '4']
True Label:  ['e', '2', '1', '+', 'e', '3', '2', '+', 'e', '4', '3', '+', 'e', '5', '4']
Predicted label:  ['\\', 's', 'i', 'n', 'x', '-', '\\', 's', 'i', 'n', 'y', '-', '\\', 's', 'i', 'n', '(', 'x', '-', 'y', ')']
True Label:  ['\\', 's', 'i', 'n', 'x', '-', '\\', 's', 'i', 'n', 'y', '-', '\\', 's', 'i', 'n', '(', 'x', '-', 'y', ')']
Predicted label:  ['\\', 's', 'q', 'r', 't', '2', '1', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', '1']
True Label:  ['\\', 's', 'q', 'r', 't', '2', '(', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', ')']
Predicted label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '(', '1', '+', 'x', ')', '-', '1', 'x']
True Label:  ['\\', 'l', 'i',

Predicted label:  ['\\', 'l', 'i', 'm', 't', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '\\', 'c', 'o', 's', '(', '-', 'x', '2', '-', 't', ')', '-', '2', '+']
True Label:  ['\\', 'l', 'i', 'm', 't', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '\\', 'c', 'o', 's', '(', '-', 'x', '2', '-', 't', ')', '-', '2', 't']
Predicted label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '(', '1', '-', '\\', 'c', 'o', 's', 'x', ')', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')', 'x', '2', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')']
True Label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '(', '1', '-', '\\', 'c', 'o', 's', 'x', ')', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')', 'x', '2', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')']
Predicted label:  ['\\', 'i', 'n', 't', '(', '2', 'x', '-', '3', 'e', 'x', ')', 'd', 'x']
True Label:  ['\\', 'i', 'n', 't', '('

In [5]:
  if __name__ == '__main__':
    main()

Preprocessing Completed!
Epoch 0   tf.Tensor(0.14716418, shape=(), dtype=float32)
Loss: tf.Tensor(3.585783, shape=(), dtype=float32)
Epoch 1   tf.Tensor(0.6273432, shape=(), dtype=float32)
Loss: tf.Tensor(2.5960042, shape=(), dtype=float32)
Epoch 2   tf.Tensor(0.796, shape=(), dtype=float32)
Loss: tf.Tensor(2.001762, shape=(), dtype=float32)
Epoch 3   tf.Tensor(0.87391055, shape=(), dtype=float32)
Loss: tf.Tensor(1.6174055, shape=(), dtype=float32)
Epoch 4   tf.Tensor(0.91277605, shape=(), dtype=float32)
Loss: tf.Tensor(1.355676, shape=(), dtype=float32)
Epoch 5   tf.Tensor(0.9314031, shape=(), dtype=float32)
Loss: tf.Tensor(1.1688524, shape=(), dtype=float32)
Epoch 6   tf.Tensor(0.9431645, shape=(), dtype=float32)
Loss: tf.Tensor(1.028739, shape=(), dtype=float32)
Epoch 7   tf.Tensor(0.95379114, shape=(), dtype=float32)
Loss: tf.Tensor(0.9189855, shape=(), dtype=float32)
Epoch 8   tf.Tensor(0.96453744, shape=(), dtype=float32)
Loss: tf.Tensor(0.8294614, shape=(), dtype=float32)
Epoch 

Predicted label:  ['\\', 'b', 'e', 't', 'a', '(', 'a', '6', '-', '1', '7', 'C', 'd', '-', '+', ')', '1', '=', '1']
True Label:  ['f', '(', 'a', 'b', '-', ')', 'f', 'c', 'd', '-', '1', ')', '1', '=', '(']
Predicted label:  ['d', '-', 'n', '(', '1', '-', ')', 'x', 'x', 'x', '\\', 's', 'q', 'r', 't']
True Label:  ['d', '-', '\\', 'p', 'i', '(', '1', '-', ')', 'x', 'x', 'x', '\\', 's', 'q', 'r', 't']
Predicted label:  ['y', 'a', '2', '+', 'b', '=', 'x']
True Label:  ['y', 'a', '2', '+', 'b', '=', 'x']
Predicted label:  ['a', '\\', 'i', 'n', 't', 'B', '1', '1', 'y', ')', '\\', 'a', 'l', 'p', 'h', 'a', 'y', '+', '\\', 'i', 'n', 't', 'B', '2', '1', 'y', '1', 'd', 'y', '=', '\\', 'a', 'l', 'p', 'h', 'a', 'x', '\\', 'a', 'l', 'p', 'h', 'a', 'x']
True Label:  ['a', '\\', 'i', 'n', 't', '\\', 'b', 'e', 't', 'a', '1', '(', 'y', ')', 'd', 'y', '+', '\\', 'i', 'n', 't', '\\', 'b', 'e', 't', 'a', '2', '(', 'y', ')', 'd', 'y', '=', '\\', 'a', 'l', 'p', 'h', 'a', 'x', '\\', 'a', 'l', 'p', 'h', 'a', 'x'

Predicted label:  ['4', 'x', 'i', 'i']
True Label:  ['\\', 'p', 'h', 'i', 'x', 'i', 'i']
Predicted label:  ['7', '(', 'x', ')', '1', '-', 'b', '-', 'a', '=']
True Label:  ['f', '(', 'x', ')', '1', '-', 'b', '-', 'a', '=']
Predicted label:  ['z', '3', '2', 'm', '2', 'n', '2', '-', '2', 'k', '-', 'n', '2', '=', '=']
True Label:  ['z', '3', '2', '-', '2', '\\', 's', 'q', 'r', 't', '2', '-', '2', '1', '-', '\\', 's', 'q', 'r', 't', '2', '=', '=']
Predicted label:  ['a', '1', '\\', 's', 'q', 'r', 't', '1', '-', '(', '-', ')', '(', '1', '-', '\\', 's', 'q', 'r', 't', '2', ')', '=', '1', 'x', 'x', 'a', '1']
True Label:  ['a', '1', 'r', '1', '-', '(', '-', ')', '(', '1', '-', 'r', '3', ')', '=', '1', 'r', 'r', '2', 'r']
Predicted label:  ['4', 'a', '4', '5', '=']
True Label:  ['\\', 'p', 'h', 'i', 'a', '\\', 'p', 'h', 'i', 'b', '=']
Predicted label:  ['(', '(', '1', '4', 'k', '+', '1', '5', '9', ')', '-', '(', '7', '1', '6', '1', '8', '6', ')', ')', '+', '1', '6', '4', '7', '3', '2', '3', '\\'

Predicted label:  ['8', '(', 'x', ')', 'd', 'x']
True Label:  ['f', '(', 'x', ')', 'd', 'x']
Predicted label:  ['4', '7', '(', '1', '3', '\\', 't', 'h', 'e', 't', 'a', '+', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '9', '8', ')', '0', '1', '4', '\\', 'd', 'i', 'v', '=', ',']
True Label:  ['4', '7', '(', '1', '3', '6', '+', '1', '9', '8', ')', '0', '1', '4', '\\', 'd', 'i', 'v', '=', '.']
Predicted label:  ['\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '8', '+', '1', '3', '\\', 't', 'h', 'e', 't', 'a', '3', '2', '3', '7', '4']
True Label:  ['1', '8', '+', '1', '3', '6', '3', '2', '3', '7', '\\', 'l', 'e', 'q']
Predicted label:  ['z', 'd', '0', '+', '=', 'b']
True Label:  ['z', 'd', '0', '+', '=', 'b']
Predicted label:  ['y', 'y', '=']
True Label:  ['y', 'Y', '=']
Predicted label:  ['-', 'b', ')', '(', '\\', 'p', 'i', 'x']
True Label:  ['-', 'b', ')', '(', '\\', 'p', 'i', '1']
Predicted label:  ['y', '(', ')', '\\', 'a', 'l', 'p', 'h', 'a', 'n', '0', 'a', '=', '\\

Predicted label:  ['0', '2', '3']
True Label:  ['\\', 't', 'h', 'e', 't', 'a', '2', '3']
Predicted label:  ['y', 'a', ')', '4']
True Label:  ['Y', 'n', ')', '(']
Predicted label:  ['(', '0', 'x', '1', ')', '1']
True Label:  ['(', '0', 'x', '1', ')', ',']
Predicted label:  ['\\', 'b', 'e', 't', 'a', '(', 'n', ')']
True Label:  ['p', '(', 'x', ')']
Predicted label:  ['6', '+', '1', '(', '4', '9', '\\', 't', 'i', 'm', 'e', 's', '0', ')', '+', '(', '2', '4', '\\', 't', 'i', 'm', 'e', 's', '1', '3', '9', ')', ')', '1', '8', '3', '9', '0', '1', '1', '1', '2']
True Label:  ['6', '+', '(', '(', '4', '9', '\\', 't', 'i', 'm', 'e', 's', '0', ')', '+', '(', '2', '4', '\\', 't', 'i', 'm', 'e', 's', '1', '3', '9', ')', ')', '1', '8', '3', '9', '0', '1', '1', '1', '\\', 'g', 'e', 'q']
Predicted label:  ['e', '-', '1']
True Label:  ['e', '-', '1']
Predicted label:  ['(', 'n', '+', '2', ')', '3', '-', '1', 'n', '-', '1', ')', '3', '0', '=']
True Label:  ['(', 'x', '+', '2', ')', '3', '-', '(', 'x', '-

...

Predicted label:  ['a', '(', 'n', ')', '=', '\\', 's', 'u', 'm', 'n', 'k', '=', '1', '(', '-', '(', ')', 'n', '-', 'k', 'k', '!']
True Label:  ['a', '(', 'n', ')', '=', '\\', 's', 'u', 'm', 'n', 'k', '=', '1', '(', '-', '1', ')', 'n', '-', 'k', 'k', '!']
Predicted label:  ['\\', 'i', 'n', 't', '2', '1', '(', '-', 'x', '2', '-', '1', 'x', '2', ')', 'e', 'x', '+', '-', '1', 'x', 'd', 'x']
True Label:  ['\\', 'i', 'n', 't', '2', '1', '(', '-', 'x', '2', '-', '1', 'x', '2', ')', 'e', 'x', '+', '-', '1', 'x', 'd', 'x']
Predicted label:  ['\\', 's', 'i', 'n', '(', '\\', 'a', 'l', 'p', 'h', 'a', '+', '\\', 'b', 'e', 't', 'a', ')', '\\', 's', 'i', 'n', '(', '\\', 'a', 'l', 'p', 'h', 'a', '\\', 's', 'q', 'r', 't', '\\', 'b', 'e', 't', 'a', ')', '=', '\\', 's', 'i', 'n', '2', '\\', 'a', 'l', 'p', 'h', 'a', '-', '\\', 's', 'i', 'n', '2', '\\', 'b', 'e', 't', 'a']
True Label:  ['\\', 's', 'i', 'n', '(', '\\', 'a', 'l', 'p', 'h', 'a', '+', '\\', 'b', 'e', 't', 'a', ')', '\\', 's', 'i', 'n', '(', '\

Predicted label:  ['\\', 's', 'i', 'n', 'x', '-', '\\', 's', 'i', 'n', 'y', '-', '\\', 's', 'i', 'n', '(', 'x', '-', 'y', ')']
True Label:  ['\\', 's', 'i', 'n', 'x', '-', '\\', 's', 'i', 'n', 'y', '-', '\\', 's', 'i', 'n', '(', 'x', '-', 'y', ')']
Predicted label:  ['\\', 's', 'q', 'r', 't', '2', '(', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '(', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', ')']
True Label:  ['\\', 's', 'q', 'r', 't', '2', '(', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', ')']
Predicted label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '(', '1', '+', 'x', ')', '-', '1', 'x']
True Label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '(', '1', '+', 'x', ')', '-', '1', 'x']
Predicted label:  ['\\', 'l', 'i', 'm', '8', '\\', 'r', 'i', 'g', 'h', 

Predicted label:  ['\\', 'i', 'n', 't', '-', '\\', 'p', 'i', '2', '0', '\\', '{', '(', '\\', 'c', 'o', 's', 'x', '+', 'e', 'x', ')', '-', '(', '\\', 't', 'h', 'e', 't', 'a', 'x', '-', '\\', 'c', 'o', 's', 'x', ')', '\\', '}', 'd', 'x']
True Label:  ['\\', 'i', 'n', 't', '-', '\\', 'p', 'i', '2', '0', '\\', '{', '(', '\\', 'c', 'o', 's', 'x', '+', 'e', 'x', ')', '-', '(', 'e', 'x', '-', '\\', 'c', 'o', 's', 'x', ')', '\\', '}', 'd', 'x']
Predicted label:  ['[', '-', '2', '3', '(', '1', '+', '-', '9', '4', 'x', ')', '-', '3', '2', '-', '4', '9', ']', '4', '0']
True Label:  ['[', '-', '2', '3', '(', '1', '+', '-', '9', '4', 'x', ')', '-', '3', '2', '-', '4', '9', ']', '4', '0']
Predicted label:  ['\\', 'l', 'i', 'm', '8', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '1', '\\', 'l', 'o', 'g', 'a', 'C', '(', '+', '2', ')', '-', '1', '2']
True Label:  ['\\', 'l', 'i', 'm', 'z', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '1', '\\', 'l', 'o', 'g', 'a

Predicted label:  ['\\', 's', 'q', 'r', 't', '2', '1', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', '1']
True Label:  ['\\', 's', 'q', 'r', 't', '2', '(', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 's', 'i', 'n', 'x', '+', '-', '1', '\\', 's', 'q', 'r', 't', '2', '\\', 'c', 'o', 's', 'x', ')']
Predicted label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '(', '1', '+', 'x', ')', '-', '1', 'x']
True Label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '(', '1', '+', 'x', ')', '-', '1', 'x']
Predicted label:  ['\\', 'l', 'i', 'm', '+', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '\\', 'c', 'o', 's', '(', '-', 'x', '2', '-', '+', ')', '-', '2', 't']
True Label:  ['\\', 'l', 'i', 'm', 't', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '\\', 'c', 'o', 's', '(', '-', '

Predicted label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '(', '1', '-', '\\', 'c', 'o', 's', 'x', ')', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')', 'x', '2', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')']
True Label:  ['\\', 'l', 'i', 'm', 'x', '\\', 'r', 'i', 'g', 'h', 't', 'a', 'r', 'r', 'o', 'w', '0', '-', '(', '1', '-', '\\', 'c', 'o', 's', 'x', ')', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')', 'x', '2', '(', '1', '+', '\\', 'c', 'o', 's', 'x', ')']
Predicted label:  ['\\', 'i', 'n', 't', '(', '2', 'x', '-', '3', 'e', 'x', ')', 'd', 'x']
True Label:  ['\\', 'i', 'n', 't', '(', '2', 'x', '-', '3', 'e', 'x', ')', 'd', 'x']
Predicted label:  ['-', '1', 'a', 'F', '(', 'a', 'x', '+', 'b', ')', '+', 'C']
True Label:  ['-', '1', 'a', 'F', '(', 'a', 'x', '+', 'b', ')', '+', 'C']
Predicted label:  ['\\', 'i', 'n', 't', '-', 'n', '2', '0', '\\', '{', '(', '\\', 'c', 'o', 's', 'x', '+', 'e', 'x', ')', '-', '(', 'e', 'x', '-', '\\', 'c', '